In [8]:
#Vibrato tech task notebook
import pandas as pd
import numpy as np
import sqlite3
import csv
import urllib
from zipfile import ZipFile

In [9]:
#Retrieve, save and unzip file
url = "https://s3-ap-southeast-2.amazonaws.com/vibrato-data-test-public-datasets/world-food-facts.zip"
urllib.request.urlretrieve (url, "world-food-facts.zip")
zf = ZipFile("world-food-facts.zip")
zf.extractall(path = './')
zf.close()

In [12]:
foodTSV = 'en.openfoodfacts.org.products.tsv'

In [13]:
#open database file and csv reader (formatted for reading tsv)

SQLITE_FILE = "openFoodFacts.db"
conn = sqlite3.connect(SQLITE_FILE) #opens sqlite and database file
myCursor = conn.cursor() #provides connection to database

reader = csv.reader(open(foodTSV), delimiter='\t')
headers = reader.__next__()
print(len(headers))

162


In [15]:
# Create table in sqlite3.  First 64 entries have type text, the rest as real numbers.  

do_this_sqlite_1 = "CREATE TABLE IF NOT EXISTS FoodFacts ( "

for headerName in headers[:64]:
    do_this_sqlite_1 += " " + headerName + " text,"

for headerName in headers[64:]:
    do_this_sqlite_1 += " " + headerName + " real,"
    
do_this_sqlite_1 = do_this_sqlite_1[:-1]

do_this_sqlite_1 += ");"

do_this_sqlite_1 = do_this_sqlite_1.replace('-', '_') #sql doesn't seem to like hyphens

# print(do_this_sqlite_1)

myCursor.execute(do_this_sqlite_1)

In [14]:
#Insert values from tsv file into table

do_this_sqlite = "INSERT INTO FoodFacts VALUES ( "
do_this_sqlite += "?," * len(headers)
do_this_sqlite = do_this_sqlite[:-1]
do_this_sqlite += ")"

print(do_this_sqlite)

rowErrors = 0

for row in reader:
    try:
        myCursor.execute(do_this_sqlite,row)
    except:
        rowErrors += 1
        
print("rowErrors = {}".format(rowErrors))
    
conn.commit()

INSERT INTO FoodFacts VALUES ( ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
rowErrors = 48


In [98]:
#PEANUT BUTTER TASK Open sqlite file and retrieve peanut butter information - List the Top 5 Peanut Butters based in 
#Australia and sorted by highest Energy content per 100g

#note - i am getting erroneous value for Choc Peanut Butter - need to investigate.

SQLITE_FILE = "openFoodFacts.db"
conn = sqlite3.connect(SQLITE_FILE) #opens sqlite and database file
peanut_frame = pd.read_sql_query('''
SELECT product_name, energy_100g, origins, manufacturing_places, purchase_places, countries 
FROM FoodFacts
''', conn)
#ORDER BY energy_100g DESC
#peanut_frame.head(2)
peanut_frame = peanut_frame[peanut_frame['product_name'].str.contains("peanut butter", case=False, na=False)]
peanut_frame = peanut_frame[peanut_frame['countries'].str.contains("Australia", case=False, na=False)]
peanut_frame = peanut_frame.sort_values(by=['energy_100g'], ascending=False)
peanut_frame.head(10)
# conn.close()

,product_name,energy_100g,origins,manufacturing_places,purchase_places,countries
332239,Choc Peanut Butter,397,,Australia,"Melbourne,Victoria,Australia",Australia
332525,Peanut Butter Crunchy - No added Salt,2670,Australia,Australia,"NSW,Australia",Australia
333127,Ridiculously Delicious Peanut Butter Super Smooth,2600,"Kingaroy,Queensland,Australia",,"Melbourne,Victoria,Australia",Australia
332049,natural peanut butter crunchy,2560,,,"Brisbane,Australia",Australia
187095,Bramwells Peanut Butter Crunchy,2530,Argentina,Argentina,"NSW,Australia",Australia
184505,Bramwells American Style Peanut Butter,2470,United States,United States,"Kempsey,NSW,Australia",Australia
331533,Smooth Peanut Butter,2380,"Imported Unknown,India",India,"Sydney,NSW",Australia
29220,Reeses Peanut Butter Cups,,United States,United States,,Australia
187207,American Style Peanut Butter Crunchy,,United States,United States,"Wyoming,NSW,Australia",Australia
331531,Crunchy Peanut Butter,,China,China,"NSW,Australia",Australia


In [49]:
#ZINC TASK Open sqlite file and retrieve zinc information - list the Top 10 Countries together with the number of 
#products with Zinc content above ‘0.001’ and that have more than one product

SQLITE_FILE = "openFoodFacts.db"
conn = sqlite3.connect(SQLITE_FILE) #opens sqlite and database file
zinc_frame = pd.read_sql_query('''SELECT countries, COUNT(product_name)
FROM FoodFacts 
WHERE zinc_100g IS NOT ""
AND zinc_100g IS NOT NULL 
AND zinc_100g > 0.001
GROUP BY countries
HAVING COUNT(product_name) > 1
ORDER BY COUNT(product_name) DESC
LIMIT 10''', conn)
zinc_frame.head(20)

,countries,COUNT(product_name)
0,US,2714
1,France,149
2,Suisse,54
3,United States,52
4,Deutschland,39
5,Canada,10
6,España,10
7,en:US,10
8,Россия,5
9,"Belgique,France",4


In [16]:
# Grouping product categories by those that contain Chicken, Pork and Tofu list their Average, 
# Median and Standard Deviation Protein content per 100g, excluding data that is not available
#(NaN)

SQLITE_FILE = "openFoodFacts.db"
conn = sqlite3.connect(SQLITE_FILE) #opens sqlite and database file
protein_frame = pd.read_sql_query('''SELECT proteins_100g, ingredients_text
FROM FoodFacts
WHERE proteins_100g IS NOT ""
AND proteins_100g IS NOT "NaN" 
AND proteins_100g IS NOT NULL 
''', conn)

In [91]:
meatDict = {}

chicken_frame = protein_frame[protein_frame['ingredients_text'].str.contains('chicken', case=False, na=False)]
pork_frame = protein_frame[protein_frame['ingredients_text'].str.contains('pork', case=False, na=False)]
tofu_frame = protein_frame[protein_frame['ingredients_text'].str.contains('tofu', case=False, na=False)]

meatDict['chicken'] = chicken_frame
meatDict['pork'] = pork_frame
meatDict['tofu'] = tofu_frame

for k, frame in meatDict.items():
    print("Average Protein content per 100g for {} products is: {}".format(k, frame['proteins_100g'].mean().round(2)))
    print("Median Protein content per 100g for {} products is: {:0.2f}".format(k, frame['proteins_100g'].median()))
    print("Std Deviation for Protein content per 100g for {} products is: {}".format(k, frame['proteins_100g'].std().round(2)))


# pork_frame.head(20)

Average Protein content per 100g for chicken products is: 10.06
Median Protein content per 100g for chicken products is: 9.73
Std Deviation for Protein content per 100g for chicken products is: 7.07
Average Protein content per 100g for pork products is: 16.93
Median Protein content per 100g for pork products is: 14.68
Std Deviation for Protein content per 100g for pork products is: 11.16
Average Protein content per 100g for tofu products is: 10.86
Median Protein content per 100g for tofu products is: 9.78
Std Deviation for Protein content per 100g for tofu products is: 7.0
